In [2]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

# Verify Spark session creation
spark.sql("SHOW DATABASES").show()

25/04/24 07:33:03 WARN Utils: Your hostname, codespaces-785606 resolves to a loopback address: 127.0.0.1; using 10.0.2.119 instead (on interface eth0)
25/04/24 07:33:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/workspaces/learn_iceberg/iceberg_pyspark_project/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0205e18a-7bcf-473a-911a-c1e9db315cd8;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
:: resolution report :: resolve 217ms :: artifacts dl 8ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apa

+---------+
|namespace|
+---------+
|  default|
+---------+



In [2]:
# Read data from the sample file
df = spark.read.csv("/workspaces/learn_iceberg/iceberg_pyspark_project/customers-5L.csv", header=True, inferSchema=True)

25/04/21 10:37:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
# check columns
df.columns

['Index',
 'Customer Id',
 'First Name',
 'Last Name',
 'Company',
 'City',
 'Country',
 'Phone 1',
 'Phone 2',
 'Email',
 'Subscription Date',
 'Website']

In [3]:
# Rename all the columns
df = df.withColumnRenamed("Customer Id", "Customer_Id")
df = df.withColumnRenamed("First Name", "First_Name")
df = df.withColumnRenamed("Last Name", "Last_Name")
df = df.withColumnRenamed("Phone 1", "Phone_1")
df = df.withColumnRenamed("Phone 2", "Phone_2")
df = df.withColumnRenamed("Subscription Date", "Subscription_Date")

In [9]:
# Create iceberg table with limmited number of records.
df.limit(5000).writeTo("local.schema.customers_5k") \
  .using("iceberg") \
  .createOrReplace()

25/04/17 10:30:52 WARN HadoopTableOperations: Error reading version hint file spark-warehouse/iceberg/schema/customers_5k/metadata/version-hint.text
java.io.FileNotFoundException: File spark-warehouse/iceberg/schema/customers_5k/metadata/version-hint.text does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:160)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.iceberg.hadoop.HadoopTableOperations.findVersion(HadoopTableOperations.java:318)
	at org.apache.iceberg.

In [10]:
# Select data from the created table without partition
spark.sql("select * from local.schema.customers_5k where month(Subscription_Date) = '2'").show()

+-----+---------------+----------+----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer_Id|First_Name| Last_Name|             Company|              City|             Country|             Phone_1|             Phone_2|               Email|Subscription_Date|             Website|
+-----+---------------+----------+----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|   51|803F5AaF0DdeaB1|  Lorraine|     Wyatt|      Levine-Jimenez|        Gateshaven|United Arab Emirates|001-712-043-1865x...|     +1-527-236-9562|uwarren@cameron.info|       2020-02-02|https://www.pache...|
|  103|44FdbD9aDDbea9B|     Diane|     Wolfe|    Peterson-Schmitt|    Port Gordonton|               Malta| +1-858-307-2119x750|+1-377-124-3750x4...|andresdodson@kha

In [11]:
# Explain the plan for the above query
spark.sql("explain select * from local.schema.customers_5k where month(Subscription_Date) = '2'").explain(mode="cost")

== Optimized Logical Plan ==
CommandResult [plan#257], Execute ExplainCommand, [[== Physical Plan ==
*(1) Filter (isnotnull(Subscription_Date#272) AND (month(Subscription_Date#272) = 2))
+- *(1) ColumnarToRow
   +- BatchScan local.schema.customers_5k[Index#262, Customer_Id#263, First_Name#264, Last_Name#265, Company#266, City#267, Country#268, Phone_1#269, Phone_2#270, Email#271, Subscription_Date#272, Website#273] local.schema.customers_5k (branch=null) [filters=Subscription_Date IS NOT NULL, groupedBy=] RuntimeFilters: []

]], Statistics(sizeInBytes=28.0 B)
   +- ExplainCommand 'Project [*], SimpleMode

== Physical Plan ==
CommandResult [plan#257]
   +- Execute ExplainCommand
         +- ExplainCommand 'Project [*], SimpleMode




In [20]:
# Create iceberg table with partition
spark.sql("""
CREATE TABLE local.schema.customers_partitioned_by_date(
  Index INT,
  Customer_Id STRING,
  First_Name STRING,
  Last_Name STRING,
  Company STRING,
  City STRING,
  Country STRING,
  Phone_1 STRING,
  Phone_2 STRING,
  Email STRING,
  Subscription_Date DATE,
  Website STRING
)
USING iceberg
PARTITIONED BY (months(Subscription_Date))
LOCATION 'spark-warehouse/iceberg/schema/customers_partitioned_by_date'
TBLPROPERTIES (
  'format' = 'iceberg/parquet',
  'format-version' = '2',
  'write.parquet.compression-codec' = 'zstd'
)
""")

DataFrame[]

In [7]:
# write data to partitioned table
df.limit(5000).writeTo("local.schema.customers_partitioned_by_date").append()

In [11]:
# Execute
spark.sql("SELECT * FROM local.schema.customers_partitioned_by_date WHERE Subscription_Date = '2021'").show()


+-----+---------------+----------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer_Id|First_Name|Last_Name|             Company|               City|             Country|             Phone_1|             Phone_2|               Email|Subscription_Date|             Website|
+-----+---------------+----------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    2|6EDdBA3a2DFA7De|    Yvonne|     Shaw|     Jensen and Sons|          Janetfort|Palestinian Terri...|          9610730173|   531-482-3000x7085|  kleinluis@vang.com|       2021-01-01|https://www.paul....|
| 1405|dD7012CFFa1dbED|    Steven|   Dunlap|        Horne-Conway|  New Autumnborough|            Guernsey|    133.063.0367x635|        711.847.2122|  ehorne@henson.

In [10]:
# Explain the plan for the above query
spark.sql("explain select * from local.schema.customers_partitioned_by_date where Subscription_Date = '2021'").explain(mode="cost")

== Optimized Logical Plan ==
CommandResult [plan#382], Execute ExplainCommand, [[== Physical Plan ==
*(1) Filter (Subscription_Date#397 = 2021-01-01)
+- *(1) ColumnarToRow
   +- BatchScan local.schema.customers_partitioned_by_date[Index#387, Customer_Id#388, First_Name#389, Last_Name#390, Company#391, City#392, Country#393, Phone_1#394, Phone_2#395, Email#396, Subscription_Date#397, Website#398] local.schema.customers_partitioned_by_date (branch=null) [filters=Subscription_Date IS NOT NULL, Subscription_Date = 18628, groupedBy=] RuntimeFilters: []

]], Statistics(sizeInBytes=28.0 B)
   +- ExplainCommand 'Project [*], SimpleMode

== Physical Plan ==
CommandResult [plan#382]
   +- Execute ExplainCommand
         +- ExplainCommand 'Project [*], SimpleMode




In [39]:
# Explain the plan for the above query
spark.sql("explain select * from local.schema.customers_partitioned_by_date where month(Subscription_Date) = '02'").explain(mode="cost")

== Optimized Logical Plan ==
CommandResult [plan#1269], Execute ExplainCommand, [[== Physical Plan ==
*(1) Filter (month(Subscription_Date#1284) = 2)
+- *(1) ColumnarToRow
   +- BatchScan local.schema.customers_partitioned_by_date[Index#1274, Customer_Id#1275, First_Name#1276, Last_Name#1277, Company#1278, City#1279, Country#1280, Phone_1#1281, Phone_2#1282, Email#1283, Subscription_Date#1284, Website#1285] local.schema.customers_partitioned_by_date (branch=null) [filters=Subscription_Date IS NOT NULL, groupedBy=] RuntimeFilters: []

]], Statistics(sizeInBytes=28.0 B)
   +- ExplainCommand 'Project [*], SimpleMode

== Physical Plan ==
CommandResult [plan#1269]
   +- Execute ExplainCommand
         +- ExplainCommand 'Project [*], SimpleMode




In [40]:
spark.sql("explain select * from local.schema.customers_partitioned_by_date WHERE Subscription_Date >= '2021-02-01' AND Subscription_Date < '2021-03-01'").explain(mode="cost")


== Optimized Logical Plan ==
CommandResult [plan#1303], Execute ExplainCommand, [[== Physical Plan ==
*(1) ColumnarToRow
+- BatchScan local.schema.customers_partitioned_by_date[Index#1308, Customer_Id#1309, First_Name#1310, Last_Name#1311, Company#1312, City#1313, Country#1314, Phone_1#1315, Phone_2#1316, Email#1317, Subscription_Date#1318, Website#1319] local.schema.customers_partitioned_by_date (branch=null) [filters=Subscription_Date IS NOT NULL, Subscription_Date >= 18659, Subscription_Date < 18687, groupedBy=] RuntimeFilters: []

]], Statistics(sizeInBytes=28.0 B)
   +- ExplainCommand 'Project [*], SimpleMode

== Physical Plan ==
CommandResult [plan#1303]
   +- Execute ExplainCommand
         +- ExplainCommand 'Project [*], SimpleMode




In [30]:
spark.sql("DESCRIBE FORMATTED local.schema.customers_partitioned_by_date").show(truncate=False)


+------------------+-----------------------------------+-------+
|col_name          |data_type                          |comment|
+------------------+-----------------------------------+-------+
|Index             |int                                |NULL   |
|Customer_Id       |string                             |NULL   |
|First_Name        |string                             |NULL   |
|Last_Name         |string                             |NULL   |
|Company           |string                             |NULL   |
|City              |string                             |NULL   |
|Country           |string                             |NULL   |
|Phone_1           |string                             |NULL   |
|Phone_2           |string                             |NULL   |
|Email             |string                             |NULL   |
|Subscription_Date |date                               |NULL   |
|Website           |string                             |NULL   |
|                  |     

In [5]:
# Create iceberg table with TRUNCATE transformation
spark.sql("""
CREATE TABLE local.schema.customers_partitioned_by_name (
  Index INT,
  Customer_Id STRING,
  First_Name STRING,
  Last_Name STRING,
  Company STRING,
  City STRING,
  Country STRING,
  Phone_1 STRING,
  Phone_2 STRING,
  Email STRING,
  Subscription_Date DATE,
  Website STRING
)
USING iceberg
PARTITIONED BY (truncate(Last_Name,5))
LOCATION 'spark-warehouse/iceberg/schema/customers_partitioned_by_name'
TBLPROPERTIES (
  'format' = 'iceberg/parquet',
  'format-version' = '2',
  'write.parquet.compression-codec' = 'zstd'
)
""")

DataFrame[]

In [4]:
df.limit(200).writeTo("local.schema.customers_partitioned_by_name").append()

In [5]:
spark.sql("select * from local.schema.customers_partitioned_by_name where Last_Name = 'Abbott'").show()

+-----+---------------+----------+---------+------------+----------+--------------------+--------------------+---------------+--------------------+-----------------+------------------+
|Index|    Customer_Id|First_Name|Last_Name|     Company|      City|             Country|             Phone_1|        Phone_2|               Email|Subscription_Date|           Website|
+-----+---------------+----------+---------+------------+----------+--------------------+--------------------+---------------+--------------------+-----------------+------------------+
|  101|dda3Da93Db3fA71|    Gordon|   Abbott|Wall-Bradley|Vernonfurt|United Arab Emirates|+1-587-566-4321x2456|+1-849-626-5599|cristina14@miller...|       2020-06-23|http://butler.com/|
+-----+---------------+----------+---------+------------+----------+--------------------+--------------------+---------------+--------------------+-----------------+------------------+



In [6]:
spark.sql("explain select * from local.schema.customers_partitioned_by_name where Last_Name = 'Abbott'").explain(mode = "cost")

== Optimized Logical Plan ==
CommandResult [plan#257], Execute ExplainCommand, [[== Physical Plan ==
*(1) Filter (Last_Name#265 = Abbott)
+- *(1) ColumnarToRow
   +- BatchScan local.schema.customers_partitioned_by_name[Index#262, Customer_Id#263, First_Name#264, Last_Name#265, Company#266, City#267, Country#268, Phone_1#269, Phone_2#270, Email#271, Subscription_Date#272, Website#273] local.schema.customers_partitioned_by_name (branch=null) [filters=Last_Name IS NOT NULL, Last_Name = 'Abbott', groupedBy=] RuntimeFilters: []

]], Statistics(sizeInBytes=28.0 B)
   +- ExplainCommand 'Project [*], SimpleMode

== Physical Plan ==
CommandResult [plan#257]
   +- Execute ExplainCommand
         +- ExplainCommand 'Project [*], SimpleMode




In [4]:
spark.sql("""
CREATE TABLE local.schema.customers_partitioned_by_bucket (
  Index INT,
  Customer_Id STRING,
  First_Name STRING,
  Last_Name STRING,
  Company STRING,
  City STRING,
  Country STRING,
  Phone_1 STRING,
  Phone_2 STRING,
  Email STRING,
  Subscription_Date DATE,
  Website STRING
)
USING iceberg
PARTITIONED BY (bucket(15,Customer_Id))
LOCATION 'spark-warehouse/iceberg/schema/customers_partitioned_by_bucket'
TBLPROPERTIES (
  'format' = 'iceberg/parquet',
  'format-version' = '2',
  'write.parquet.compression-codec' = 'zstd'
)
""")

DataFrame[]

In [5]:
df.limit(500).writeTo("local.schema.customers_partitioned_by_bucket").append()

In [6]:
spark.sql("select * from local.schema.customers_partitioned_by_bucket ").show(10)

+-----+---------------+----------+---------+--------------------+-----------------+--------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer_Id|First_Name|Last_Name|             Company|             City|       Country|             Phone_1|             Phone_2|               Email|Subscription_Date|             Website|
+-----+---------------+----------+---------+--------------------+-----------------+--------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|   14|bB7E28Aa85cbDD3|  Jonathan|    Morse|         Potter-Holt|       Mcgeeburgh|Western Sahara|    054.140.9190x178|   461-138-3043x6409|tami64@rice-mclau...|       2021-06-20|https://www.patri...|
|   28|2F1E68Ed8ecDA2a|  Stefanie|   Gibson|       Wheeler Group|       Dillonside|       Armenia|       (246)767-7316|          8183240781|   edward62@rios.net|       2021-04-03|https://www.wells

In [7]:
spark.sql("explain select * from local.schema.customers_partitioned_by_bucket where Customer_Id = 'b9Da13bedEc47de'").explain(mode = "cost")

== Optimized Logical Plan ==
CommandResult [plan#258], Execute ExplainCommand, [[== Physical Plan ==
*(1) Filter (Customer_Id#264 = b9Da13bedEc47de)
+- *(1) ColumnarToRow
   +- BatchScan local.schema.customers_partitioned_by_bucket[Index#263, Customer_Id#264, First_Name#265, Last_Name#266, Company#267, City#268, Country#269, Phone_1#270, Phone_2#271, Email#272, Subscription_Date#273, Website#274] local.schema.customers_partitioned_by_bucket (branch=null) [filters=Customer_Id IS NOT NULL, Customer_Id = 'b9Da13bedEc47de', groupedBy=] RuntimeFilters: []

]], Statistics(sizeInBytes=28.0 B)
   +- ExplainCommand 'Project [*], SimpleMode

== Physical Plan ==
CommandResult [plan#258]
   +- Execute ExplainCommand
         +- ExplainCommand 'Project [*], SimpleMode




In [8]:
spark.sql("""
CREATE TABLE local.schema.customers_partitioned_by_country_city (
  Index INT,
  Customer_Id STRING,
  First_Name STRING,
  Last_Name STRING,
  Company STRING,
  City STRING,
  Country STRING,
  Phone_1 STRING,
  Phone_2 STRING,
  Email STRING,
  Subscription_Date DATE,
  Website STRING
)
USING iceberg
PARTITIONED BY (Country)
LOCATION 'spark-warehouse/iceberg/schema/customers_partitioned_by_country_city'
TBLPROPERTIES (
  'format' = 'iceberg/parquet',
  'format-version' = '2',
  'write.parquet.compression-codec' = 'zstd'
)
""")

DataFrame[]

In [9]:
df.limit(5000).writeTo("local.schema.customers_partitioned_by_country_city").append()

In [10]:
spark.sql("ALTER TABLE local.schema.customers_partitioned_by_country_city ADD PARTITION FIELD City").show()

++
||
++
++



In [4]:
df.limit(100).writeTo("local.schema.customers_partitioned_by_country_city").append()

In [5]:
spark.sql("ALTER TABLE local.schema.customers_partitioned_by_country_city ADD PARTITION FIELD year(Subscription_Date)").show()

++
||
++
++



In [6]:
df.limit(100).writeTo("local.schema.customers_partitioned_by_country_city").append()

In [12]:
spark.sql("SELECT * FROM local.schema.customers_partitioned_by_country_city.snapshots").show(truncate=False)


+-----------------------+-------------------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                                     |summary                                                                                                                                                                                                                                                 

In [14]:
spark.sql("SELECT * FROM local.schema.customers_partitioned_by_country_city VERSION AS OF 396991467446470913;").show()

+-----+---------------+----------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer_Id|First_Name| Last_Name|             Company|                City|Country|             Phone_1|             Phone_2|               Email|Subscription_Date|             Website|
+-----+---------------+----------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-----------------+--------------------+
|  157|8dBcFDF5ED9D5D2|    Evelyn|   Krueger|     Kelley and Sons|     East Coreyburgh|Lesotho|   (872)960-8087x638|001-729-045-6337x...|emmacameron@sloan...|       2021-04-24|http://www.valenz...|
|  236|c0Ea931f3eBb4A5|     Jerry|     Evans|        Velez-Keller|   Port Tracyborough|Lesotho|    785.352.8872x837|+1-848-210-2482x7907|kingkaitlin@mccal...|       2021-12-18|https://www.farre...|
|  342|3fb

In [16]:
spark.sql("SELECT * FROM local.schema.customers_partitioned_by_country_city TIMESTAMP AS OF '2025-04-21 10:29:54.179';").show()

+-----+---------------+----------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer_Id|First_Name| Last_Name|             Company|                City|Country|             Phone_1|             Phone_2|               Email|Subscription_Date|             Website|
+-----+---------------+----------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-----------------+--------------------+
|  157|8dBcFDF5ED9D5D2|    Evelyn|   Krueger|     Kelley and Sons|     East Coreyburgh|Lesotho|   (872)960-8087x638|001-729-045-6337x...|emmacameron@sloan...|       2021-04-24|http://www.valenz...|
|  236|c0Ea931f3eBb4A5|     Jerry|     Evans|        Velez-Keller|   Port Tracyborough|Lesotho|    785.352.8872x837|+1-848-210-2482x7907|kingkaitlin@mccal...|       2021-12-18|https://www.farre...|
|  342|3fb

In [17]:
spark.sql("SELECT * FROM local.schema.customers_partitioned_by_country_city.history;").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2025-04-21 10:29:...| 396991467446470913|               NULL|               true|
|2025-04-21 10:38:...|7386602405767638480| 396991467446470913|               true|
|2025-04-21 10:44:...|1590301241980826574|7386602405767638480|               true|
+--------------------+-------------------+-------------------+-------------------+



Iceberg schema evolution

In [4]:
spark.sql("DESCRIBE TABLE  local.schema.customers_partitioned_by_country_city").show(truncate=False)

+-----------------+------------------------+-------+
|col_name         |data_type               |comment|
+-----------------+------------------------+-------+
|Index            |int                     |NULL   |
|Customer_Id      |string                  |NULL   |
|First_Name       |string                  |NULL   |
|Last_Name        |string                  |NULL   |
|Company          |string                  |NULL   |
|City             |string                  |NULL   |
|Country          |string                  |NULL   |
|Phone_1          |string                  |NULL   |
|Phone_2          |string                  |NULL   |
|Email            |string                  |NULL   |
|Subscription_Date|date                    |NULL   |
|Website          |string                  |NULL   |
|                 |                        |       |
|# Partitioning   |                        |       |
|Part 0           |Country                 |       |
|Part 1           |City                    |  

In [ ]:
# Reorder columns in iceberg
spark.sql("ALTER TABLE local.schema.customers_partitioned_by_country_city ALTER COLUMN City AFTER Customer_Id;").show(truncate=False)

In [ ]:
# Add column
spark.sql("""
    ALTER TABLE local.schema.customers_partitioned_by_country_city
    ADD COLUMN referrer STRING AFTER Website
""")


In [ ]:
# Remove column
spark.sql("""
    ALTER TABLE local.schema.customers_partitioned_by_country_city 
    DROP COLUMN referrer
""")


In [ ]:
# Rename column
spark.sql("""
    ALTER TABLE local.schema.customers_partitioned_by_country_city 
    RENAME COLUMN Website TO Website_name
""")

In [ ]:
# Change data type
spark.sql("""
    ALTER TABLE local.schema.customers_partitioned_by_country_city
    ALTER COLUMN subscription_date SET DATA TYPE TIMESTAMP
""")